In [1]:
import numpy as np

In [2]:
#Function for Gini calculation
def Gini(classes):
    
    size = len(classes)
    
    class_set = set(classes)
    class_count = {}
    for cur in class_set:
        class_count[cur] = 0
        
    for cur in classes:
        class_count[cur] += 1.0
    
    g_metric = 0
    for cur in class_count.keys():
        pk = class_count[cur] / size
        g_metric += pk * (1-pk)
        
    return g_metric

In [3]:
#Function for IG calculation
def IG(values, classes, split, curr_Gini):
    left_vals = []
    left_class = []
    right_vals = []
    right_class = []
    
    for idx in range(len(values)):
        if values[idx] <= split:
            left_vals.append(values[idx])
            left_class.append(classes[idx])
        else:
            right_vals.append(values[idx])
            right_class.append(classes[idx])
            
#     print(Gini(left_class))
#     print(Gini(right_class))
    IG_val = curr_Gini - len(left_vals) * Gini(left_class)/len(values) - len(right_vals) * Gini(right_class) / len(values)
    return IG_val

In [4]:
#Selecting best split for one column
def FindBestSplit(values, classes):
    init_Gini = Gini(classes)
    best_IG = -100000000
    
    splits = []
    
    unique_vals = sorted(set(values))
    if len(unique_vals) <= 1:
        return unique_vals, best_IG
    
    for idx in range(len(unique_vals)-1):
        splits.append((unique_vals[idx]+unique_vals[idx+1])/2)
        
    for cur in splits:
        cur_IG = IG(values, classes, cur, init_Gini)
        if cur_IG > best_IG:
            best_IG = cur_IG
            best_split = cur
            
    return best_split, best_IG

In [5]:
#SImple tree class for our decision tree
class DT():
    
    def __init__(self, param_num, gini, split):
        self.left = None
        self.right = None
        self.param_num = param_num
        self.gini = gini
        self.split = split
        self.predict_class = -1

#Print function - just for debug
    def print_tree(self):
        print(self.gini, self.param_num, self.split, self.predict_class)
        
        if self.left:
            self.left.print_tree()
            
        if self.right:
            self.right.print_tree()

In [6]:
#Main clasifier
class DT_classifier():
    
    def __init__(self, max_depth=-1):
        self.max_depth = max_depth
#         self.tree = DT()

#Internal fitting function
    def dt_fit(self, X, y, cur_depth, max_depth):
    
        best_IG = -1000000000
        features_num = X.shape[1]
        left_X = []
        left_y = []
        right_X = []
        right_y = []

        #If Gini == 0 - no more splits required
        if Gini(y) == 0:
            result = DT(0, 0, 0)
            result.predict_class = y[0]
            return result
        
        #Checking for max depth and returning most probable class if exceed
        if cur_depth >= max_depth and max_depth != -1:
            class_set = set(y)
            class_count = {}
            for cur in class_set:
                class_count[cur] = 0

            for cur in y:
                class_count[cur] += 1.0
                
            best_class_count = 0
            for key, val in class_count.items():
                if val > best_class_count:
                    return_class = key
            result = DT(0, 0, 0)
            result.predict_class = return_class
            return result

        #Checking all columns for best split
        for i in range(features_num):
            cur_split, cur_IG = FindBestSplit(X[:,i], y)
            if cur_IG > best_IG:
                best_IG = cur_IG
                best_split = cur_split
                best_feature = i

#         print(best_IG, best_split, best_feature)

        #Creating left & right arrays according to best split selection
        for i in range(X.shape[0]):
            if X[i][best_feature] < best_split:
                left_X.append(X[i])
                left_y.append(y[i])
            else:
                right_X.append(X[i])
                right_y.append(y[i])

#         print(np.array(left_X))
#         print(np.array(left_X).shape)
#         print(np.array(right_X))
#         print(np.array(right_X).shape)
        #Recursion!!
        result = DT(best_feature, best_IG, best_split)
        result.left = self.dt_fit(np.array(left_X), np.array(left_y), cur_depth+1, max_depth)
        result.right = self.dt_fit(np.array(right_X), np.array(right_y), cur_depth+1, max_depth)

        return result
    
    #Wrapper function for fit - need to update class information
    def fit(self, X, y):
        self.tree = self.dt_fit(X, y, 0, self.max_depth)
    
    #Internal fit function - just walk through the tree. Stop if Gini == 0
    def dt_predict(self, x, dtree):
        if dtree.gini == 0:
            return dtree.predict_class

        if x[dtree.param_num] < dtree.split:
            return self.dt_predict(x, dtree.left)
        else:
            return self.dt_predict(x, dtree.right)
    
    #Wrapper function for internal predict
    def predict(self, x):
        return self.dt_predict(x, self.tree)

In [7]:
A = np.array([[1,1], [1,2], [1,3], [2,1], [2,2], [2,3], [3,1], [3,2], [3,3]])
A

array([[1, 1],
       [1, 2],
       [1, 3],
       [2, 1],
       [2, 2],
       [2, 3],
       [3, 1],
       [3, 2],
       [3, 3]])

In [8]:
y = np.array([1, 1, 2, 0, 1, 1, 4, 0, 1])

In [13]:
model = DT_classifier(max_depth=2)

In [14]:
model.fit(A, y)

In [15]:
model.predict([3,0])

4

In [16]:
model.tree.print_tree()

0.06172839506172839 0 2.5 -1
0.08333333333333337 1 1.5 -1
0 0 0 1
0 0 0 2
0.3333333333333334 1 1.5 -1
0 0 0 4
0 0 0 1
